## Implementeer elk gemaakt ETL-schema in Python

**imports & connection**

In [3]:
import pandas as pd
import pyodbc


In [4]:
DB = {'servername': 'LAPTOP-LPE28RPE\SQLEXPRESS', 
    'database': 'United_outdoors'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                              ';DATABASE=' + DB['database'])

export_cursor = export_conn.cursor()
export_cursor   

**Database connection**

In [5]:

Adventure = {
    'servername' : 'LAPTOP-LPE28RPE\SQLEXPRESS',
    'database' : 'AdventureWorks2019'
}


Northwind = {
    'servername' : 'LAPTOP-LPE28RPE\SQLEXPRESS',
    'database' : 'Northwind'
}

access_db_path = r'C:\Users\Humberto de Castro\OneDrive\Desktop\SEM4\AenC\aenc.accdb'

#Connect to AdventureWorks
Adventure_conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={Adventure['servername']};DATABASE={Adventure['database']};Trusted_Connection=yes;")
Adventure_cursor = Adventure_conn.cursor()

#Connect to Northwind
Northwind_conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={Northwind['servername']};DATABASE={Northwind['database']};Trusted_Connection=yes;")
Northwind_cursor = Northwind_conn.cursor()

#Connect to AenC
AenC_conn = pyodbc.connect(f"DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={access_db_path};")



In [6]:
AdventureWorks_queries = {
    'SalesOrderHeader' : 'SELECT * FROM Sales.SalesOrderHeader',
    'SalesOrderDetail' : 'SELECT * FROM Sales.SalesOrderDetail'
}

Northwind_queries = {
    'Orders' : 'SELECT * FROM Orders',
    'OrderDetails' : 'SELECT * FROM [Order Details]'
}

AenC_queries = {
    'sales_order' : 'SELECT * FROM sales_order',
    'sales_order_item' : 'SELECT * FROM sales_order_item'
}

dataframes = {}

# Lees elke tabel in een DataFrame

for table_name, query in AdventureWorks_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, Adventure_conn)

for table_name, query in Northwind_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, Northwind_conn)

for table_name, query in AenC_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, AenC_conn)

#als je ik elk tabel als een dataframe/ variabele wil behandelen of aanroepen moet ik dit uitvoeren.
for table_name, df in dataframes.items():
    globals()[table_name] = df


C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_12272\3695189965.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table_name] = pd.read_sql_query(query, Adventure_conn)
C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_12272\3695189965.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table_name] = pd.read_sql_query(query, Northwind_conn)
C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_12272\3695189965.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table

### Order

**Extract**

In [7]:
#eerst horizontaal samenvoegen per database:
AdventureWorks_Order = pd.merge(SalesOrderHeader, SalesOrderDetail, on = 'SalesOrderID')
Northwind_Order = pd.merge(Orders, OrderDetails, on = 'OrderID')
AenC_Order = pd.merge(sales_order, sales_order_item, on = 'id')

**Transform**

**Check of orderID's conflicten**

In [8]:
AdventureWorks_Order.rename(columns={'SalesOrderID': 'OrderID'}, inplace=True)

order_order_ids = set(AdventureWorks_Order['OrderID'])
order_details_order_ids = set(Northwind_Order['OrderID'])
Order_AenC_ids = set(AenC_Order['id'])

# Zoek conflicten tussen AdventureWorks en Northwind
conflicting_order_ids_1 = order_order_ids.intersection(order_details_order_ids)

# Zoek conflicten tussen AdventureWorks en AenC
conflicting_order_ids_2 = order_order_ids.intersection(Order_AenC_ids)

# Zoek conflicten tussen Northwind en AenC
conflicting_order_ids_3 = order_details_order_ids.intersection(Order_AenC_ids)

# Combineer alle conflicterende ID's
all_conflicting_order_ids = conflicting_order_ids_1.union(conflicting_order_ids_2).union(conflicting_order_ids_3)

if all_conflicting_order_ids:
    print("Conflicting Order IDs found:", all_conflicting_order_ids)
else:
    print("No conflicts found in Order IDs.")

# Display the conflicting Order IDs (if any)
conflicts_df = pd.DataFrame(list(all_conflicting_order_ids), columns=['Conflicting Order IDs'])
print(conflicts_df)

No conflicts found in Order IDs.
Empty DataFrame
Columns: [Conflicting Order IDs]
Index: []


In [18]:


AdventureWorks_Order.rename(columns={
    'SalesOrderID': 'OrderID',
    'SalesOrderDetailID': 'OrderDetailID',
    'OrderQty': 'Quantity',
    'UnitPriceDiscount': 'Discount',
    'Order_Date': 'OrderDate'
}, inplace=True)

Northwind_Order.rename(columns={
    'ShipVia': 'ShipMethodID'
})

AenC_Order.rename(columns={
    'id': 'OrderID',
    'item_id': 'OrderDetailID',
    'qty': 'Quantity',
    'price': 'UnitPrice',
    'discount': 'Discount',
    'date': 'OrderDate',
    'customer_id': 'CustomerID',
    'employee_id': 'EmployeeID',
    'ship_method_id': 'ShipMethodID',
    'product_id': 'ProductID'
}, inplace=True)

# Zorg ervoor dat alle vereiste kolommen bestaan en selecteer ze
required_columns = ['OrderID', 'OrderDetailID', 'Quantity', 'UnitPrice', 'Discount', 'OrderDate', 'CustomerID', 'EmployeeID', 'ShipMethodID', 'ProductID']

for col in required_columns:
    if col not in AdventureWorks_Order.columns:
        AdventureWorks_Order[col] = None
    if col not in Northwind_Order.columns:
        Northwind_Order[col] = None
    if col not in AenC_Order.columns:
        AenC_Order[col] = None

# Selecteer de kolommen in de juiste volgorde
AdventureWorks_Order = AdventureWorks_Order[required_columns]
Northwind_Order = Northwind_Order[required_columns]
AenC_Order = AenC_Order[required_columns]

# Combineer de dataframes
combined_df = pd.concat([AdventureWorks_Order, Northwind_Order, AenC_Order], ignore_index=True)

# Toon het resultaat
print(combined_df)

       OrderID OrderDetailID Quantity  UnitPrice  Discount  OrderDate  \
0        43659             1        1   2024.994       0.0 2011-05-31   
1        43659             2        3   2024.994       0.0 2011-05-31   
2        43659             3        1   2024.994       0.0 2011-05-31   
3        43659             4        1   2039.994       0.0 2011-05-31   
4        43659             5        1   2039.994       0.0 2011-05-31   
...        ...           ...      ...        ...       ...        ...   
124570    2652          None     None        NaN       NaN        NaT   
124571    2653          None     None        NaN       NaN        NaT   
124572    2654          None     None        NaN       NaN        NaT   
124573    2654          None     None        NaN       NaN        NaT   
124574    2654          None     None        NaN       NaN        NaT   

       CustomerID EmployeeID ShipMethodID ProductID  
0           29825       None            5       776  
1           298

C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_12272\2734654936.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([AdventureWorks_Order, Northwind_Order, AenC_Order], ignore_index=True)
